# RECORD LINKAGE AND MEASUREMENT
By Maryah Garner

## Table of Contents
* [Set Up Environment](#set-up)
* [Organization names ](#orgs)
* [PI Names](#pi_names)
 * [Cleaning PI Names](##clean_names)
* [Develop Measures ](#measures)
* [Checkpoint 1](#checkpoint1)
* [Link Data on PI Name ](#link_name)
* [Develop measures for change over time ](#change_measures)
* [Results](#results)
* [Checkpoint 2](#checkpoint2)

We are going to read in projects data from 2015, create measures, and then link it to the 'DF_2017.csv' data that has already been prepared for you using the RePORTER_PRJ_C_FY2017_new data. This is a non-standard approach to performing analytics for data with the same structure over different years. 

We are using this approach to give you the extremely valuable experience of linking data without overwhelming you with different data sources. You will be able to use the code and skills developed in this notebook to link data from different sources in the future. 

## Set Up Environment  <a class="anchor" id="set-up"></a>

In [ ]:
# general use imports
import pandas as pd

### Read in Project Data

In [ ]:
# Specify a path with the data folder
# Change "NAME" to your name as recorded on your computer
# path = 'C:/Users/NAME/PADM-GP_2505/Data/'
Path = '---/PADM-GP_2505/Data'

#### Read in Pre-processed 2017 Data 
First, we will read in the Preprocessed 2017 (`DF_2017.csv`) data. If you still need to, make sure you download this data from Britespace and put it in your `PADM-GP_2505/Data` folder.

In [ ]:
# Read-in the DF_2017 CSV file 
DF_2017 = pd.read_csv(Path + '/DF_2017.csv')

# View the first 5 observations 
DF_2017.head(5)

#### Read in 2015  Projects Data
Next, we will read in the raw projects data from 2015. We will clean this data and use it to develop a data frame that has one observation per PI (for PIs from NYU), similar to the `DF_2017` seen above. 

In [ ]:
# Read-in a CSV file
grants_2015 = pd.read_csv(Path + '/Projects/RePORTER_PRJ_C_FY2015.csv', encoding='latin-1')

# View the first 5 observations 
grants_2015.head()

### Organization Names  <a class="anchor" id="orgs"></a>
We want to see how the organizations are recorded in the data

First, we will select variables that will help us understand better how organizations are recorded in the data and then look at the data.

In [ ]:
#View the first 5 observations for select varables 
grants_2015[['APPLICATION_ID', 'ORG_DUNS', 'ORG_IPF_CODE', 'ORG_NAME', 'ORG_STATE']].head(5)



#### Subsetting string
We will subset the grants_2015 for projects from NYU
   - In order to use the str.contains function, we first need to convert the `ORG_NAME` variable into a string

In [ ]:
# Convert ORG_NAME into a string variable
grants_2015['ORG_NAME'] = grants_2015['ORG_NAME'].astype(str)

In [ ]:
# Subsetting by a keyword in a string using str.contains()
# Select all observations that have "NEW YORK UNIVERSITY" in the organization name.
df_NYU = grants_2015[grants_2015['ORG_NAME'].str.contains('NEW YORK UNIVERSITY', na = False)]


# View the first 2 observations 
df_NYU.head(2)

In [ ]:
# View the first 5 observations of select variables
df_NYU[['APPLICATION_ID', 'ORG_DUNS', 'ORG_IPF_CODE', 'ORG_NAME', 'ORG_STATE']].head(5)

As you can see NEW YORK UNIVERSITY and NEW YORK UNIVERSITY SCHOOL OF MEDICINE are considered different organizations in this data. Depending on your research question, it might be more appropriate to consider them as the same organization. You will find many organizations exhibit simular patterns, so you need to be thoughtful as to what you consider to be a single organization in your research projects. 

We will group ORG_NAME to get the number of Projects that were awarded for each. This will allow us to see the names of all ORG_NAMEs that have "NEW YORK UNIVERSITY" in its name. In this case there are only 2, but some organizations have more then two ORG_NAMEs which might not all displayed using the head command, so you need to be careful.

In [ ]:
# calculate how many projects (unique application ids) that were awarded by each ORG_NAME
df_NYU.groupby('ORG_NAME')['APPLICATION_ID'].nunique().sort_values(ascending=False)

#### Replace ORG_NAME 
For the purpose of this notebook, we will treat both NEW YORK UNIVERSITY and NEW YORK UNIVERSITY SCHOOL OF MEDICINE	as the same organization.

We will create a list of possible names for NEW YORK UNIVERSITY and then replace all Org names in that list with 'NEW YORK UNIVERSITY'.


In [ ]:
# Save a list of possible names for the organization (NYU in this case)
possible_NYU_names = ['NEW YORK UNIVERSITY', 'NEW YORK UNIVERSITY SCHOOL OF MEDICINE']

# Replace any instance you see a string in the list of possible_org_names with 'NEW YORK UNIVERSITY'
df_NYU = df_NYU.replace(possible_NYU_names, 'NEW YORK UNIVERSITY')

# recalculate how many projects that were awarded by each ORG_NAME
df_NYU.groupby('ORG_NAME')['APPLICATION_ID'].nunique().sort_values(ascending=False)

In your projects, you can create a list like we did for NYU, for each group of ORG_NAMEs you want to consolidate into a single organization. Then use the `replace` function for each of your created lists. 

## PI Names <a class="anchor" id="pi_names"></a>
Recall our goal is to develop an analytic dataframe that we can link to the `DF_2017` data, linking on PI_NAME. To do this, we need to understand how the PI names are recorded. 
- We will begin by counting the number of projects for each of the unique `PI_NAMEs`. We will then carefully examine the `PI_NAMEs` in the output.

In [ ]:
# Calculate how many Projects each unique observation for PI_NAMEs has
df_NYU_CI = df_NYU.groupby(['PI_NAMEs','PI_IDS'])['APPLICATION_ID'].nunique().sort_values(ascending=False).to_frame().reset_index()

# View the first 10 observations
df_NYU_CI.head(10)

As you can see from the output in row seven above (or if you read the codebook), sometimes more than one PI exists in an award. Thus we cannot treat each unique value recorded as the PI_NAMEs as a unique PI. We will look at all of the projects OGEDEGBE, OLUGBENGA G is a PI on, to exemplify this further.

In [ ]:
# Show full text in a cell
pd.set_option('display.max_colwidth', -1)

# Show all PI_NAMEs which contain name "OLUGBENGA"
df_NYU_CI[df_NYU_CI['PI_NAMEs'].str.contains('OLUGBENGA', na = False)]


As you can see OGEDEGBE, OLUGBENGA G was a PI on 11 projects, not just the two projects that are associated with the string that only contains his name

### Cleaning PI Names <a class="anchor" id="clean_names"></a>
To clean the PI names, we will split up the names when more than one is recorded and then create a new column with a single PI for each observation.

#### Transform each element of a list-like to a row, replicating index values
We will create a temporary dataframe with an index number and a single PI name for each observation. The index number will be repeated for each PI that is split up from a list of PIs in the PI_NAMEs variable.
- We will use the str.split function to split up the PI names, so we first need to convert PI_NAMEs into a string variable
- We will then use the `str.split` function to separate the list of names into a different element for each PI. We are splitting the string at each `;` 
- We use the `explode` function to transform each element into a row, replicating index values

In [ ]:
# First we need to convert ORG_NAME into a string variable
df_NYU['PI_NAMEs'] = df_NYU['PI_NAMEs'].astype(str)

In [ ]:
# Make a temperary dataframe that creates an observation for each PI. 
# Use the explode function to split the PI_Names at the ;
temp = df_NYU['PI_NAMEs'].str.split(';').explode().reset_index()

# Rename the PI_NAMEs variable
temp = temp.rename(columns = {'PI_NAMEs': 'PI_NAME'})

# look at the first 5 observations
temp.head()

#### Remove observations with empty strings 
Because we split the PI_NAMEs using a `;`, and there is a `;` at the end of every PI (even those without a name proceeding it in the string), we created many observations with an empty string for PI_NAME. We will proceed with removing the empty string.

In [ ]:
# Only keep observations in the temp dataframe where PI_NAME is not an empty string
temp = temp[temp['PI_NAME'] !=""]

# View the first 4 observations 
temp.head(4)

#### Remove undesired section of the string
We are almost there, however we still need to remove the ` (contact)` part of the string from some of the names; otherwise PI_names like `DAW, NATHANIEL DOUGLASS (contact)` will not link to `DAW, NATHANIEL DOUGLASS` 

In [ ]:
# For the the PI_NAME variable, use the str.replace fundtion to replace ` (contact)` with nothing 
temp['PI_NAME'] = temp['PI_NAME'].str.replace(' \(contact\)', '')

temp.head()

#### Merge dataframes
Now that we have a clean list of PIs, we will merge this temporary dataframe with df_NYU. After doing so, we will see the new PI_NAME variable as well as the old PI_NAMEs variable. Note all other variables will repeat for each of the separated PI names. 

First, let's look at the df_NYU dataframe to make sure we are ready to merge on the index.

In [ ]:
# view the select variables for the firts 4 observations
df_NYU[['APPLICATION_ID', 'PI_NAMEs','FULL_PROJECT_NUM', 'PROJECT_TITLE','PROJECT_START', 'PROJECT_END', 'TOTAL_COST']].head()

##### reset the index
Note that the index reflects the column number from the full dataframe, and the column number lines up with the index numbers from the temperary dataframe. Thus, we need to reset the index before merging.

In [ ]:
# Reset the index
df_NYU2 = df_NYU.reset_index()

# view the select variables for the firts 4 observations
df_NYU2[['index', 'APPLICATION_ID', 'PI_NAMEs','FULL_PROJECT_NUM', 'PROJECT_TITLE','PROJECT_START', 'PROJECT_END', 'TOTAL_COST']].head()

In [ ]:
# Merge this temperary dataframe with df_NYU

df_NYU3 = df_NYU2.merge(temp, on = 'index')


# view the select variables for the firts 4 observations
df_NYU3[['APPLICATION_ID', 'PI_NAMEs', 'PI_NAME','FULL_PROJECT_NUM', 'PROJECT_TITLE','PROJECT_START', 'PROJECT_END', 'TOTAL_COST']].head()

# Develop Measures <a class="anchor" id="measures"></a>
Using the 2015 data, we want to create measures at the PI level that will allow us to research how the `Cancer Moonshot` influenced the people working on cancer research. 

#### All Projects 
- The number of projects 
- The total value all of projects

#### All NCI Projects 
- Number of projects from the National cancer institute
- Value of projects from the National cancer institute

#### New NCI Projects
- Number of new projects from the National cancer institute
- Value of new projects from the National cancer institute

#### Most Common CI for each PI? 


## All Projects

### Number of Projects

In [ ]:
# Calculate how many Projects each PI has
PI = df_NYU3.groupby(['PI_NAME'])['APPLICATION_ID'].nunique().sort_values(ascending=False)

# Convert into a dataframe and reset index
PI = PI.to_frame().reset_index()

# Rename APPLICATION_ID to Total_Projects_2015
PI.rename(columns={'APPLICATION_ID':'Total_Projects_2015'}, inplace = True)

# View the first 2 observations 
PI.head(2)

### Total Value of  All Projects

In [ ]:
# Calculate the sum of the total costs for each PI
Value_PI = df_NYU3.groupby('PI_NAME')['TOTAL_COST'].sum()

# Convert into a dataframe and reset index
Value_PI = Value_PI.to_frame().reset_index()

# Rename TOTAL_COST to Total_Cost_All_2015
Value_PI.rename(columns={'TOTAL_COST':'Total_Cost_All_2015'}, inplace = True)

# View the first 2 observations 
Value_PI.head(2)

In [ ]:
# Look at how many rows and columns there are
Value_PI.shape

In [ ]:
# Marge together  `PI` and `Value_PI on PI_NAME`, creating a new data frame called `DF_2015.`
# Use an outer merge. 
DF_2015 = pd.merge(PI, Value_PI, on='PI_NAME', how = 'outer')

# Look at how many rows and columns there are
DF_2015.shape

In [ ]:
# View the first 5 observations 
DF_2015.head()

## All Projects from the National Cancer Institute¶

### Number of Projects from the National Cancer Institute 

In [ ]:
# Use a conditional statement do create a new dataframe were there are only projects where the IC_NAME is NATIONAL CANCER INSTITUTE. 
NCI = df_NYU3[df_NYU3['IC_NAME'] == 'NATIONAL CANCER INSTITUTE']
NCI.shape

In [ ]:
NCI.to_csv(Path + "example_data.csv", encoding='utf8')

In [ ]:
# Calculate how many NCI Projects each PI has
NCI_PI = NCI.groupby(['PI_NAME'])['APPLICATION_ID'].nunique().sort_values(ascending=False)

# Convert into a dataframe and reset index
NCI_PI = NCI_PI.to_frame().reset_index()

# Let's correct the columns names, this shouldn't be APPLICATION_ID but Total_NCI_Projects_2015
NCI_PI.rename(columns={'APPLICATION_ID':'Total_NCI_Projects_2015'}, inplace = True)

# View the first 5 observations 
NCI_PI.head()

In [ ]:
# Look at how many rows and columns there are before merging 
NCI_PI.shape

In [ ]:
# Merge NCI_PI into the DF_2015 data frame 
DF_2015 = pd.merge(DF_2015, NCI_PI, on='PI_NAME', how = 'outer')

# Look at how many rows and columns there are after merging 
DF_2015.shape

Note, we used an outer merge, so we are maintaining the total number of observations from the `DF_2015` data frame. If we had used an `inner merge,` we would only capture the PIs in both dataframes. This would be bad, since it would be dropping all the PIs who are not on an NCI grant.

### Total value of National Cancer Instute Projects

In [ ]:
# Calculate the sum of the total NCI costs for each PI
NCI_Value = NCI.groupby('PI_NAME')['TOTAL_COST'].sum().sort_values(ascending=False)

# Convert into a dataframe and reset index
NCI_Value = NCI_Value.to_frame().reset_index()

# Rename the colums New_TOTAL_COST
NCI_Value.rename(columns={'TOTAL_COST':'NCI_Total_Cost_2015'}, inplace=True)

# View the first 3 observations
NCI_Value.head(3)

In [ ]:
# Look at how many rows and columns there are before merging 
NCI_Value.shape

In [ ]:
# Merge the NCI_Value into the DF_2015 data frame 
DF_2015 = pd.merge(DF_2015, NCI_Value, on='PI_NAME', how = 'outer')

# Look at how many rows and columns there are after merging 
DF_2015.shape

In [ ]:
# View the first 3 observations
DF_2015.head(3)

## New grants from the National Cancer Institute¶

#### New projects
We need to identify all projects that started this year we will do this in 3 steps 
1. Convert PROJECT_START into a date variable
2. Use the project start date to create a `project_start_year` column
3. Create a new data frame that only has a observations where the `PROJECT_START_YEAR` is 2015

In [ ]:
# convert PROJECT_START into a date variable
NCI['PROJECT_START'] = pd.to_datetime(NCI['PROJECT_START'])

# Create a new year variable from the PROJECT_START
NCI['PROJECT_START_YEAR'] = NCI['PROJECT_START'].dt.year

# View the first 3 observarions for select variables 
NCI[['PROJECT_START_YEAR', 'PROJECT_START']].head(3)

In [ ]:
# Create a new data frame that has a variable where the PROJECT_START_YEAR is 2015
NCI_new = NCI[NCI['PROJECT_START_YEAR'] == 2015]

# View the first 3 observations for select variables 
NCI_new[['PROJECT_START_YEAR', 'PROJECT_START']].head(3)

### Number of New Grants from the National Cancer Institute 

In [ ]:
# Calculate how many new NCI Projects each PI has
NCI_new_PI = NCI_new.groupby(['PI_NAME'])['APPLICATION_ID'].nunique().sort_values(ascending=False)

# Convert into a dataframe and reset index
NCI_new_PI = NCI_new_PI.to_frame().reset_index()

# Let's correct the columns names, this shouldn't be APPLICATION_ID but Total_New_NCI_Projects_2015
NCI_new_PI.rename(columns={'APPLICATION_ID':'Total_New_NCI_Projects_2015'}, inplace = True)

# View the first 3 observations
NCI_new_PI.head(3)

In [ ]:
# Look at how many rows and columns there are before merging 
NCI_new_PI.shape

In [ ]:
DF_2015 = pd.merge(DF_2015, NCI_new_PI, on='PI_NAME', how = 'outer')

# Look at how many rows and columns there are after merging 
DF_2015.shape

In [ ]:
# Look at the first 3 observations 
DF_2015.head(3)

# Check point 1: (Assignment 3 due March 10th)  <a class="anchor" id="checkpoint1"></a>
Add a column to the `DF_2015` dataframe that has the total value of New National Cancer Instute Projects (3points)
Due 

### What Is the Most Common CI for each PI? 

In [ ]:
# Calculate how many projects each PI has with each CI
temp = df_NYU3.groupby(['PI_NAME', 'IC_NAME'])['APPLICATION_ID'].nunique().sort_values(ascending=False)

# Convert into a dataframe and reset index
temp = temp.to_frame().reset_index()

# Rename APPLICATION_ID Count
temp.rename(columns={'APPLICATION_ID':'Count'}, inplace = True)

# Group by PI_NAME and select the resultant rows with the max value in the (numerical column -> Count)
# Sorted in ascending order of Count
temp = temp.groupby(['PI_NAME'],sort=True)[['IC_NAME', 'Count']].max()

# Rename IC_NAME Most_common_IC
temp.rename(columns={'IC_NAME':'Most_common_IC_2015', 'Count': 'Most_common_IC_Count_2015'}, inplace = True)
temp = temp.reset_index()

# View the first 3 observations
temp.head(3)

In [ ]:
# Merge the temp dataframe into the DF_2015 data frame 
# Use an outer merge
DF_2015 = pd.merge(DF_2015, temp, on='PI_NAME', how = 'outer')

# Look at how many rows and columns there are
DF_2015.shape

In [ ]:
# View the first 3 observations
DF_2015.head(3)

# Link Data on PI Name <a class="anchor" id="link_name"></a>
Now we will merge together the `DF_2015` Date we just created with the pre processed `DF_2017` we read in at the beginning of this notebook. We will use an `outer` joint, so that we capture PI's the had grant(s) in only 2015, both 2015 and 2017, and only 2017

In [ ]:
# Create a new dataframe by using an outer merge to merge the DF_2015 and DF_2017 dataframes
# merging on PI_NAME
DF = pd.merge(DF_2015, DF_2017, on='PI_NAME', how = 'outer')
DF.shape

In [ ]:
# Look at the first 5 observations
DF.head(5)

In [ ]:
# Look at the last 5 observations 
DF.tail(5)

#### Fill in the Missing values with 0
Caution! it is often not the case that you should fill missing values with Zero. It is appropriate in this case because we know for example, FALKNER ANNEGRET LEA, was not a PI on an NIH project for NYU in 2015. Note that this narrows the scope of the question, though, because FALKNER ANNEGRET LEA could have worked on non-NIH projects in 2015, or she could have worked on NIH projects at a different institution in 2015. You have to be very mindful of what assumptions you are making and how you are narrowing the scope of your research when you fill in missing values with zero for your own project. 

In [ ]:
# use the fillma function to fill all of the missing values with Zero
DF= DF.fillna(0)

# view the first 5 observations 
DF.head()

## Develop measures for change over time  <a class="anchor" id="change_measures"></a>
Now that we have brought together the 2015 and 2017 data, we can create measures for the change in outcomes. 
We will develop two measures
1. Change in the number of NCI Projects
2. Change in the Total cost of NCI Projects

#### Change in the Number of National Cancer Institute Projects

In [ ]:
# Create a new column (Change_in_NCI_Projects) that is equal to the difference in Total_NCI_Projects_2017 and Total_NCI_Projects_2015'
DF['Change_in_NCI_Projects'] = DF['Total_NCI_Projects_2017'] - DF['Total_NCI_Projects_2015']

# Look at the first 5 observations for select variables 
DF[['PI_NAME','Total_NCI_Projects_2017', 'Total_NCI_Projects_2015', 'Change_in_NCI_Projects']].head()

#### Change in the Cost of National Cancer Institute Projects

In [ ]:
# Create a new column (Change_in_NCI_Cost) that is equal to the difference in NCI_Total_Cost_2017 and NCI_Total_Cost_2015'
DF['Change_in_NCI_Cost'] = DF['NCI_Total_Cost_2017'] - DF['NCI_Total_Cost_2015']

# Look at the first 5 observations for select variables 
DF[['PI_NAME','NCI_Total_Cost_2017', 'NCI_Total_Cost_2015', 'Change_in_NCI_Cost']].head()


# Results <a class="anchor" id="results"></a>
Now that the data is prepaired, we have answer some questions about PIs in 2015 and 2017

In [ ]:
DF.columns

#### Hpw many PIs form NYU were there in 2015?

In [ ]:
# Look at the first 2 observations for select variables 
DF.head(2)

In [ ]:
# Subset dataframe for PIs who had more than zero total projects in 2015
temp = DF[DF['Total_Projects_2015'] > 0] 

# Count the number of unique PI names
temp["PI_NAME"].nunique()

#### Hpw many PIs form NYU were there in 2017?

In [ ]:
# Subset dataframe for PIs who had more than zero total projects in 2017
temp = DF[DF['Total_Projects_2017'] > 0] 

# Count the number of unique PI names
temp["PI_NAME"].nunique()

#### How many PIs from NYU had NCI projects in 2015?

In [ ]:
# Subset dataframe for PIs who had more than zero NCI projects in 2015
temp = DF[DF['Total_NCI_Projects_2015'] > 0] 

# Count the number of unique PI names
temp["PI_NAME"].nunique()

#### How many PIs from NYU had NCI projects in 2017?

In [ ]:
# Subset dataframe for PIs who had more than zero NCI projects in 2017
temp = DF[DF['Total_NCI_Projects_2017'] > 0] 

# Count the number of unique PI names
temp["PI_NAME"].nunique()

#### How many PIs from NYU had new NCI projects in 2015?

In [ ]:
# Subset dataframe for PIs who had more than zero new NCI projects in 2015
temp = DF[DF['Total_New_NCI_Projects_2015'] > 0] 

# Count the number of unique PI names
temp["PI_NAME"].nunique()

#### How many PIs from NYU had new NCI projects in 2017?

In [ ]:
# Subset dataframe for PIs who had more than zero new NCI projects in 2017
temp = DF[DF['Total_New_NCI_Projects_2017'] > 0] 

# Count the number of unique PI names
temp["PI_NAME"].nunique()

#### How many PIs only had non NCI Project(s) in 2015 but had at least one NCI project in 2017?

In [ ]:
# Subset dataframe for PIs who had zero NCI projects in 2015
# and more than zero projects in 2015
# and more than zero NCI projects in 2017
Converted_PIs = DF[(DF['Total_NCI_Projects_2015'] == 0) & 
           (DF['Total_Projects_2015'] > 0) &
           (DF['Total_NCI_Projects_2017'] > 0)]

# Count the number of unique PI names
Converted_PIs["PI_NAME"].nunique()

In [ ]:
Converted_PIs.head()

In [ ]:
# look at the first 5 observations for select variables
Converted_PIs[['PI_NAME', 'Total_Projects_2015', 'Total_Cost_All_2015', 'Total_NCI_Projects_2015', 
      'Total_NCI_Projects_2017', 'NCI_Total_Cost_2017', 'Most_common_IC_2015']].head()

# Checkpoint 2 (Assignment 3 due March 10th) <a class="anchor" id="checkpoint2"></a>

How many PIs had a NCI funded projects in 2017 but not 2015? (1point)

For those who have at least one NCI funded project in 2015, how many NCI funded projects do they have on average? (1point)

For those who have at least one NCI funded project in 2017, how many NCI funded projects do they have on average? (1point)

For the PIs who only had non-NCI Project(s) in 2015 but had at least one NCI project in 2017, what are the 5 most common ICs they are moving from (in total, not per PI)? (2points)

Can we sum the `Total_Cost_All_2015` column in the `DF` dataframe to get the total cost of projects in 2015? Why or why not? (2points)
- This is not a question you need code to answer, instead, create a Markdown cell, and answer in a complete sentence. 